In [1]:
import os
import torch
import random
import numpy as np
from PIL import Image
from glob import glob
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
# Set random seeds for reproducibility
torch.manual_seed(2024)
np.random.seed(2024)
random.seed(2024)

In [3]:
### Custom Dataset Class ###
class FireDataset(Dataset):
    """
    Custom Dataset class for loading images from 'fire' and 'no_fire' folders.
    """
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = glob(f"{root_dir}/*/*.jpg")
        
        # Define class labels
        self.class_labels = {"fire": 0, "no_fire": 1}
        self.image_paths = [p for p in self.image_paths if any(cls in p for cls in self.class_labels)]
        
    def get_class_label(self, image_path):
        """Extract class label from image path based on folder name."""
        label = os.path.basename(os.path.dirname(image_path))
        return self.class_labels[label]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        """Retrieve an image and its label."""
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.get_class_label(image_path)

        if self.transform:
            image = self.transform(image)

        return image, label


In [4]:
### Data Loading Function ###
def create_dataloaders(root_dir, transform, batch_size, split_ratios=(0.8, 0.1, 0.1), num_workers=4):
    """
    Prepare dataloaders for training, validation, and testing.
    """
    dataset = FireDataset(root_dir=root_dir, transform=transform)
    total_len = len(dataset)
    train_len = int(total_len * split_ratios[0])
    val_len = int(total_len * split_ratios[1])
    test_len = total_len - (train_len + val_len)
    
    train_set, val_set, test_set = random_split(dataset, lengths=[train_len, val_len, test_len])
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    
    return train_loader, val_loader, test_loader


In [5]:
# Set constants and prepare data
ROOT_DIR = "Datasets/corrected_wildfires_dataset" 
BATCH_SIZE = 32
IMAGE_SIZE = 224
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

transform = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=MEAN, std=STD)
])

train_loader, val_loader, test_loader = create_dataloaders(
    root_dir=ROOT_DIR,
    transform=transform,
    batch_size=BATCH_SIZE
)


In [6]:
### Simple CNN Model ###
class SimpleCNN(nn.Module):
    """
    A simple CNN model with Conv2D, ReLU, and MaxPool layers.
    """
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 56 * 56, 128)  # Adjust based on image size after conv layers
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        
        x = x.view(x.size(0), -1)  # Flatten the tensor
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [7]:
# Instantiate model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=2).to(device)

In [8]:
### Training and Validation Functions ###
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    epoch_loss, correct_preds = 0, 0
    
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct_preds += (preds == labels).sum().item()
        
    return epoch_loss / len(dataloader), correct_preds / len(dataloader.dataset)

def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    epoch_loss, correct_preds = 0, 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            epoch_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct_preds += (preds == labels).sum().item()

    return epoch_loss / len(dataloader), correct_preds / len(dataloader.dataset)

In [9]:
# Hyperparameters
NUM_EPOCHS = 10
LEARNING_RATE = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
### Training Loop ###
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")

In [ ]:
# Test the model
test_loss, test_acc = validate_epoch(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")